In [1]:
import pandas as pd
import pickle
import re
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
data=pd.read_pickle(r"C:\Users\HP\Desktop\ml project ms1\data.pkl")
data

,bathrooms,bedrooms,price_display,square_feet,state,longitude,total_numberofrooms
0,2,2,1194,6.684612,27,-78.7783,4
1,1,1,1370,6.678342,48,-89.4003,2
2,1,1,1009,6.327937,10,-82.3656,2
3,1,1,695,6.396930,29,-96.0404,2
4,3,3,3695,7.377759,5,-118.2972,6
...,...,...,...,...,...,...,...
8995,1,2,1158,6.774224,7,-72.1062,3
8996,1,1,2035,6.714171,47,-122.0000,2
8997,1,2,424,6.738152,48,-88.4411,3
8998,1,2,1417,6.192362,5,-118.0875,3


In [3]:
df=pd.read_csv(r"C:\Users\HP\Downloads\ApartmentRentPrediction.csv")

In [4]:
with open(r"C:\Users\HP\Desktop\ml project ms1\most_frequent_values_ms1 (1).pkl", 'rb') as file:
    most_frequent_values = pickle.load(file)
for col in df:
   
    df[col].fillna(most_frequent_values[col], inplace=True)

In [5]:
df.isnull().sum()

id               0
category         0
title            0
body             0
amenities        0
bathrooms        0
bedrooms         0
currency         0
fee              0
has_photo        0
pets_allowed     0
price            0
price_display    0
price_type       0
square_feet      0
address          0
cityname         0
state            0
latitude         0
longitude        0
source           0
time             0
dtype: int64

In [6]:
df['price_display'] = df['price_display'].apply(lambda x: re.sub(r'\D', '', x))
df['price_display'] = df['price_display'].astype(int)

In [7]:
with open(r"C:\Users\HP\Desktop\ml project ms1\ordinal_encoders (2).pickle", 'rb') as f:
    ordinal_encoders = pickle.load(f)

# Apply ordinal encoding to the categorical columns in the test data
for col in df.select_dtypes(include=['object']).columns:
    df[col] = ordinal_encoders.fit_transform(df[[col]].astype(str))

In [8]:
df.isnull().sum()

id               0
category         0
title            0
body             0
amenities        0
bathrooms        0
bedrooms         0
currency         0
fee              0
has_photo        0
pets_allowed     0
price            0
price_display    0
price_type       0
square_feet      0
address          0
cityname         0
state            0
latitude         0
longitude        0
source           0
time             0
dtype: int64

In [9]:
common_columns = df.columns.intersection(data.columns)
for col in df:
  if col not in common_columns  :
     df.drop(columns=col,inplace=True)
common_columns

Index(['bathrooms', 'bedrooms', 'price_display', 'square_feet', 'state',
       'longitude'],
      dtype='object')

In [10]:
df = pd.DataFrame(df, columns=data.columns)
df['total_numberofrooms']=df['bathrooms']+df['bedrooms']

In [11]:
df

,bathrooms,bedrooms,price_display,square_feet,state,longitude,total_numberofrooms
0,2.5,2.0,1194,800,26.0,-78.7783,4.5
1,1.0,1.0,1370,795,47.0,-89.4003,2.0
2,1.0,1.0,1009,560,9.0,-82.3656,2.0
3,1.0,1.0,695,600,28.0,-96.0404,2.0
4,3.0,3.0,3695,1600,4.0,-118.2972,6.0
...,...,...,...,...,...,...,...
8995,1.0,2.0,1158,875,6.0,-72.1062,3.0
8996,1.0,1.0,2035,824,46.0,-122.1507,2.0
8997,1.0,2.0,424,844,47.0,-88.4411,3.0
8998,1.0,2.0,1417,489,4.0,-118.0875,3.0


In [12]:
columns_to_check = ['bathrooms','bedrooms','total_numberofrooms']
for col in columns_to_check:

    df[col] = df[col].apply(lambda x: int(x) if pd.notnull(x) else None)
    df[col].astype(int)

In [13]:
X = df.drop(columns=['price_display'])
y = df['price_display']

In [14]:
from scipy.stats import boxcox


y, _ = boxcox(y)


In [15]:
with open(r"C:\Users\HP\Desktop\ml project ms1\scaler.pickle", 'rb') as f:
    scaler = pickle.load(f)
    X_scaled=scaler.transform(X)

In [16]:
X_scaled

array([[ 1.23739232e+00,  2.68433795e-01,  1.84454201e+03,
        -4.36720558e-02,  1.03753426e+00,  6.70102610e-01],
       [-6.30058901e-01, -7.93563333e-01,  1.83291574e+03,
         1.34910222e+00,  3.40181495e-01, -7.94004983e-01],
       [-6.30058901e-01, -7.93563333e-01,  1.28648105e+03,
        -1.17115599e+00,  8.02021781e-01, -7.94004983e-01],
       ...,
       [-6.30058901e-01,  2.68433795e-01,  1.94685319e+03,
         1.34910222e+00,  4.03154643e-01, -6.19511865e-02],
       [-6.30058901e-01,  2.68433795e-01,  1.12138802e+03,
        -1.50276892e+00, -1.54318306e+00, -6.19511865e-02],
       [ 1.23739232e+00,  2.68433795e-01,  2.46305957e+03,
        -1.50276892e+00, -1.57015936e+00,  6.70102610e-01]])

In [17]:
with open(r"C:\Users\HP\Desktop\ml project ms1\rf_regressor_model.pickle", 'rb') as f:
    loaded_model = pickle.load(f)
new_data_predictions = loaded_model.predict(X_scaled)
mse_new_data = mean_squared_error(y, new_data_predictions)
r2_new_data = r2_score(y, new_data_predictions)

print("Mean Squared Error (New Data):", mse_new_data)
print("R-squared Score (New Data):", r2_new_data)

Mean Squared Error (New Data): 0.0009152856774534476
R-squared Score (New Data): -0.12991590563592492


In [18]:
with open(r"C:\Users\HP\Desktop\ml project ms1\xgb_regressor_model.pickle", 'rb') as f:
    xgb_loaded_model = pickle.load(f)

y_test_pred_loaded = xgb_loaded_model.predict(X_scaled)

mse_test_loaded = mean_squared_error(y, y_test_pred_loaded)
r2_test_loaded = r2_score(y, y_test_pred_loaded)

print("Mean Squared Error (Test) with Loaded XGBoost Model:", mse_test_loaded)
print("R-squared Score (Test) with Loaded XGBoost Model:", r2_test_loaded)

Mean Squared Error (Test) with Loaded XGBoost Model: 0.0007548638733115626
R-squared Score (Test) with Loaded XGBoost Model: 0.06812406437108554


In [19]:
with open(r"C:\Users\HP\Desktop\ml project ms1\linear_regressor_model.pickle", 'rb') as f:
    linear_loaded_model = pickle.load(f)


y_test_pred_loaded = linear_loaded_model.predict(X_scaled)


mse_test_loaded = mean_squared_error(y, y_test_pred_loaded)
r2_test_loaded = r2_score(y, y_test_pred_loaded)

print("Mean Squared Error (Test) with Loaded Linear Regression Model:", mse_test_loaded)
print("R-squared Score (Test) with Loaded Linear Regression Model:", r2_test_loaded)

Mean Squared Error (Test) with Loaded Linear Regression Model: 4544859279895196.0
R-squared Score (Test) with Loaded Linear Regression Model: -5.610607612170142e+18
